## 라이브러리

In [4]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import csv
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'AppleGothic'
# plt.rcParams['font.family'] = 'NanumGothic'

## vvt 파일 불러오기

In [5]:
select_category = 'Drama' # 카테고리를 선택해주세요!! 


title_list = os.listdir(f'../script/{select_category}')

title_list = pd.DataFrame(title_list)
title_list = title_list.rename(columns = {0 : 'title'})

title_list

,title
0,Disenchantment
1,Sherlock
2,Peaky.Blinders
3,Black.Mirror
4,The.Good.Doctor
5,MINDHUNTER
6,Stranger.Things
7,Better.Call.Saul
8,Narcos
9,Rick.and.Morty


# 위의 리스트에서 작품 인덱스를 넣으세요

In [6]:
title_num = 13 # 작품 인덱스 넣는 곳

In [7]:
file_list = os.listdir(f'../script/{select_category}/' + title_list.loc[title_num]['title'])
file_list

['Anne.with.an.E.S01E06.WEBRip.Netflix.en[cc].vtt',
 'Anne.with.an.E.S01E01.WEBRip.Netflix.en[cc].vtt',
 'Anne.with.an.E.S01E04.WEBRip.Netflix.en[cc].vtt',
 'Anne.with.an.E.S01E03.WEBRip.Netflix.en[cc].vtt',
 'Anne.with.an.E.S01E02.WEBRip.Netflix.en[cc].vtt',
 'Anne.with.an.E.S01E05.WEBRip.Netflix.en[cc].vtt',
 'Anne.with.an.E.S01E07.WEBRip.Netflix.en[cc].vtt']

In [8]:
title = title_list.loc[title_num]['title']

path = f'../script/{select_category}/' + title_list.loc[title_num]['title']

full_segments = ''

for file_name in file_list:
    
    opened_file = open(f'{path}/{file_name}', encoding='utf8')
    content = opened_file.read()
    segments = content.split('NOTE /SegmentIndex')[1]    # 한줄씩 정리

    full_segments += segments

segments = full_segments.split('\n\n')

In [9]:
full_segments

'\n\n                                           \n\n                    \n1\n00:00:06.131 --> 00:00:09.467  position:50.00%,middle  align:middle size:80.00%  line:84.67% \n<c.bg_transparent>[pants]</c.bg_transparent>\n\n2\n00:00:17.600 --> 00:00:20.395  position:50.00%,middle  align:middle size:80.00%  line:84.67% \n<c.bg_transparent>[groans, pants]</c.bg_transparent>\n\n3\n00:00:40.373 --> 00:00:41.374  position:50.00%,middle  align:middle size:80.00%  line:84.67% \n<c.bg_transparent>Help me!</c.bg_transparent>\n\n4\n00:00:42.792 --> 00:00:44.169  position:50.00%,middle  align:middle size:80.00%  line:84.67% \n<c.bg_transparent>Please help me!</c.bg_transparent>\n\n5\n00:00:44.252 --> 00:00:45.879  position:50.00%,middle  align:middle size:80.00%  line:84.67% \n<c.bg_transparent>[pants]</c.bg_transparent>\n\n6\n00:00:47.964 --> 00:00:49.007  position:50.00%,middle  align:middle size:80.00%  line:84.67% \n<c.bg_transparent>Anne!</c.bg_transparent>\n\n7\n00:00:49.507 --> 00:00:51.509  p

## 불필요한 부분 제거

In [10]:
m = re.compile(r"\<.*?\>")
def clean(content):
    if '♪' in content: return None
    if '[' in content: return None
    if ']' in content: return None
    if '(' in content: return None
    if ')' in content: return None
    if '{' in content: return None
    if '}' in content: return None
    new_content = m.sub('',content)
    new_content = new_content.replace('-->','')
    remove_point_1 = new_content.find('position:')
    remove_point_2 = new_content.find('% \n')
    new_content = new_content[:remove_point_1] + new_content[remove_point_2+3:]
    return new_content

In [11]:
new_segments = [clean(s) for s in segments if len(s)!=0][4:]
new_segments[:20]

['4\n00:00:42.792  00:00:44.169  Please help me!',
 None,
 '6\n00:00:47.964  00:00:49.007  Anne!',
 "7\n00:00:49.507  00:00:51.509  -Diana?\n-Minnie May, she's sick.",
 '8\n00:00:51.593  00:00:54.220  -We need to wake Miss Cuthbert.\n-She went to see the premier.',
 '9\n00:00:54.304  00:00:56.264  -Did your parents go as well?\n-Yes!',
 "10\n00:00:57.682  00:00:59.017  What's wrong with Minnie May?",
 "11\n00:00:59.100  00:01:01.519  I don't know.\nShe's choking and coughing. It's horrible!",
 '12\n00:01:02.187  00:01:03.188  Croup.',
 '13\n00:01:03.271  00:01:04.647  We need the doctor.',
 None,
 '15\n00:01:09.569  00:01:11.780  Put this on you\nbefore you catch your death.',
 "16\n00:01:11.863  00:01:14.949  Aunt Josephine is staying with us,\nbut she doesn't know what to do.",
 "17\n00:01:15.033  00:01:17.952  -She's never seen a cough like this.\n-I have.",
 '18\n00:01:19.037  00:01:22.415  Matthew, ride to Carmody and fetch\nthe doctor. Minnie May is gravely ill.',
 "19\n00:01:22.

## 줄번호 / 속도 시작 / 속도 끝 / 자막 - 데이터 분리

In [12]:
trimmed_segments = []
for segment in new_segments:
    if segment != None and segment != '':
      split_segment = segment.split()
      times = split_segment[:3]
      script = split_segment[3:]
      times.append(script)
      trimmed_segments.append(times)

In [13]:
trimmed_segments

[['4', '00:00:42.792', '00:00:44.169', ['Please', 'help', 'me!']],
 ['6', '00:00:47.964', '00:00:49.007', ['Anne!']],
 ['7',
  '00:00:49.507',
  '00:00:51.509',
  ['-Diana?', '-Minnie', 'May,', "she's", 'sick.']],
 ['8',
  '00:00:51.593',
  '00:00:54.220',
  ['-We',
   'need',
   'to',
   'wake',
   'Miss',
   'Cuthbert.',
   '-She',
   'went',
   'to',
   'see',
   'the',
   'premier.']],
 ['9',
  '00:00:54.304',
  '00:00:56.264',
  ['-Did', 'your', 'parents', 'go', 'as', 'well?', '-Yes!']],
 ['10',
  '00:00:57.682',
  '00:00:59.017',
  ["What's", 'wrong', 'with', 'Minnie', 'May?']],
 ['11',
  '00:00:59.100',
  '00:01:01.519',
  ['I',
   "don't",
   'know.',
   "She's",
   'choking',
   'and',
   'coughing.',
   "It's",
   'horrible!']],
 ['12', '00:01:02.187', '00:01:03.188', ['Croup.']],
 ['13', '00:01:03.271', '00:01:04.647', ['We', 'need', 'the', 'doctor.']],
 ['15',
  '00:01:09.569',
  '00:01:11.780',
  ['Put', 'this', 'on', 'you', 'before', 'you', 'catch', 'your', 'death.']],
 [

## 데이터 저장

In [14]:
file_name = file_list[0].split('.S01')[0] + '.WEBRip.Netflix.en[cc]'
file_name

'Anne.with.an.E.WEBRip.Netflix.en[cc]'

In [15]:


with open(f'{path}/{file_name}.csv', 'w', encoding='utf8', newline='') as f:
    for line in trimmed_segments:
        thewriter = csv.writer(f)
        thewriter.writerow(line)

## 최종 - 줄번호 / 속도 시작 / 속도 끝 / 대사 별로 컬럼 분리

In [16]:
data = pd.read_csv(f'{path}/{file_name}.csv', names=['num', 'start', 'end', 'script'])

data

,num,start,end,script
0,4,00:00:42.792,00:00:44.169,"['Please', 'help', 'me!']"
1,6,00:00:47.964,00:00:49.007,['Anne!']
2,7,00:00:49.507,00:00:51.509,"['-Diana?', '-Minnie', 'May,', ""she's"", 'sick.']"
3,8,00:00:51.593,00:00:54.220,"['-We', 'need', 'to', 'wake', 'Miss', 'Cuthber..."
4,9,00:00:54.304,00:00:56.264,"['-Did', 'your', 'parents', 'go', 'as', 'well?..."
...,...,...,...,...
4072,654,00:42:53.487,00:42:55.573,"[""It's"", 'simple', 'fare,', 'but', 'thank', 'y..."
4073,655,00:42:55.656,00:42:58.367,"['I', 'wonder', ""what's"", 'happened', 'to', 't..."
4074,657,00:43:00.119,00:43:01.620,"['That', 'must', 'be', 'him.']"
4075,658,00:43:03.414,00:43:04.456,"[""I'll"", 'get', 'it.']"


In [17]:
data.to_csv(f'{path}/{file_name}.csv', columns=['start', 'end', 'script'], index=False)
data = pd.read_csv(f'{path}/{file_name}.csv')

data

,start,end,script
0,00:00:42.792,00:00:44.169,"['Please', 'help', 'me!']"
1,00:00:47.964,00:00:49.007,['Anne!']
2,00:00:49.507,00:00:51.509,"['-Diana?', '-Minnie', 'May,', ""she's"", 'sick.']"
3,00:00:51.593,00:00:54.220,"['-We', 'need', 'to', 'wake', 'Miss', 'Cuthber..."
4,00:00:54.304,00:00:56.264,"['-Did', 'your', 'parents', 'go', 'as', 'well?..."
...,...,...,...
4072,00:42:53.487,00:42:55.573,"[""It's"", 'simple', 'fare,', 'but', 'thank', 'y..."
4073,00:42:55.656,00:42:58.367,"['I', 'wonder', ""what's"", 'happened', 'to', 't..."
4074,00:43:00.119,00:43:01.620,"['That', 'must', 'be', 'him.']"
4075,00:43:03.414,00:43:04.456,"[""I'll"", 'get', 'it.']"


In [18]:
print(f'{title}/{file_name}')

Anne.with.an.E/Anne.with.an.E.WEBRip.Netflix.en[cc]


In [19]:
# colab
# data = pd.read_csv(f'{path}/dramas/Breaking Bad/Breaking.Bad.S01E01.WEBRip.Netflix.encc.csv')
# data = data.rename(columns = {'[]':'script'})
# data = data.rename(columns = {'Unnamed: 0':'num', 'Unnamed: 1':'start', 'Unnamed: 2':'end', '[]':'script'})
# data.to_csv(f'{path}/dramas/Breaking Bad/Breaking.Bad.S01E01.WEBRip.Netflix.encc.csv')

# data = pd.read_csv('./Breaking.Bad.S01E01.WEBRip.Netflix.en[cc].csv')
# data = data.rename(columns = {'Unnamed: 0':'num', 'Unnamed: 1':'start', 'Unnamed: 2':'end', '[]':'script'})
# data.to_csv('Breaking.Bad.S01E01.WEBRip.Netflix.en[cc].csv')

## 파일 불러올때 불필요한 컬럼이 포함되기 때문에 index_col=0 포함해서 불러와야 합니다.

In [20]:
# colab
# data = pd.read_csv(f'{path}/dramas/Breaking Bad/Breaking.Bad.S01E01.WEBRip.Netflix.encc.csv', index_col=0)
# data = pd.read_csv('./Breaking.Bad.S01E01.WEBRip.Netflix.en[cc].csv')
# data